In [51]:
# imports and link kismet xml files
import re, pandas as pd, xml.etree.ElementTree as ET
from math import radians, cos, sin, asin, sqrt
from IPython.display import display
import folium
from folium import FeatureGroup, LayerControl, Map, Marker

tree = ET.parse('sesh/Kismet-20210719-20-27-11-1.netxml')
gps_tree = ET.parse('sesh/Kismet-20210719-20-27-11-1.gpsxml')
root, gps_root = tree.getroot(), gps_tree.getroot()

In [52]:
# find vendor with MAC address
file = open('../mac-vendors.txt', 'r', encoding="utf8")
mac_dict = { mac[0]: mac[1] for mac in [re.split(r'\t+', line.strip()) for line in file] }
def getDeviceType(bssid): return mac_dict.get(bssid[:8]) # first 3 octets

In [53]:
# haversine formula GPS coordinate distance
def haversine(lon1, lat1, lon2, lat2): 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon, dlat = lon2-lon1 , lat2-lat1
    return (6371*(2*asin(sqrt(sin(dlat/2)**2+cos(lat1)*cos(lat2)*sin(dlon/2)**2))))/1.069

In [54]:
# netxml parser that creates array data for Wi-Fi devices
def appendArray(net, child): 
    if (child.tag == net) :
        bssid, essid = "", ""
        gps, time, encryption = [], [], [] 
        scan = 'BSSID' if net == "wireless-network" else 'client-mac'
        for bssid_tag in child.findall(scan): bssid = bssid_tag.text 
        for gps_tag in child.findall("gps-info"):
            gps = [[float(gps_tag[0].text), float(gps_tag[1].text)],[float(gps_tag[4].text), float(gps_tag[5].text)]]
        time = [child.attrib['first-time'], child.attrib['last-time']]
        for ssid_tag in child.findall('SSID'):
            for essid_tag in ssid_tag.findall('essid'): essid= essid_tag.text
            encryption = [info_tag.text for info_tag in ssid_tag if info_tag.tag == "encryption"]
        StationList.append([bssid,gps,time,getDeviceType(bssid),encryption,essid])  # add individual device entry

In [55]:
# create array of all devices found
StationList = [] # MAC, GPS[[min],[max]], time[[first seen],[last seen]], device type, encryption, ESSID
for child in root:
    appendArray("wireless-network",child) # network
    for subchild in child:
        appendArray("wireless-client", subchild) # client under network tag

### creep detecting starts here

In [56]:
# creep list with threshold of .20 miles
threshold = .50
creeps = [StationList[index] for index in range(0,len(StationList)) if haversine(StationList[index][1][0][0],StationList[index][1][0][1],StationList[index][1][1][0],StationList[index][1][1][1]) > threshold]        

# remove duplicates
unique, duplicate = [], []
for creep, i in zip(creeps, range(len(creeps))) :
    if creep[0] not in unique: unique.append(creep[0])
    else: duplicate.append(i)
for index in sorted(duplicate, reverse=True): del creeps[index]  

# check source MAC
for child in creeps : child[1] = [[float(gpss.attrib["lat"]),float(gpss.attrib["lon"])] for gpss in gps_root if (('source' in gpss.attrib) and gpss.attrib['source']==child[0])]

pd.DataFrame(creeps, columns=["BSSID", "GPS","Dates","Manufacturer","Encryption","ESSID"])

,BSSID,GPS,Dates,Manufacturer,Encryption,ESSID
0,4C:66:41:9A:81:4B,"[[33.678879, -117.603043], [33.678879, -117.60...","[Mon Jul 19 20:31:24 2021, Mon Jul 19 20:33:32...",SAMSUNG ELECTRO-MECHANICS(THAILAND),[],
1,4E:66:41:9A:81:4B,"[[33.684544, -117.619072], [33.684544, -117.61...","[Mon Jul 19 20:27:26 2021, Mon Jul 19 20:34:17...",None,"[WPA+PSK, WPA+AES-CCM]",Hot Bitch Network
2,78:0C:B8:EF:6A:05,"[[33.684544, -117.619072], [33.684544, -117.61...","[Mon Jul 19 20:27:27 2021, Mon Jul 19 20:34:17...",Intel Corporate,[],
3,8E:AA:B5:1C:31:3E,"[[33.684544, -117.619072], [33.684544, -117.61...","[Mon Jul 19 20:27:25 2021, Mon Jul 19 20:37:11...",None,"[WPA+TKIP, WPA+PSK, WPA+AES-CCM]",Bingus Headquarters
4,B8:27:EB:BE:A6:1A,"[[33.684544, -117.619072], [33.684544, -117.61...","[Mon Jul 19 20:27:26 2021, Mon Jul 19 20:37:06...",Raspberry Pi Foundation,[],
5,D8:F1:5B:10:AD:19,"[[33.684547, -117.61908], [33.684547, -117.619...","[Mon Jul 19 20:27:34 2021, Mon Jul 19 20:37:00...",None,[],


In [57]:
# this code tries to approximate a path by searching for the closest point from the starting coordinate.
# may need to be tweaked according to your data set

# gather all gps points
coordinates = list(set(map(tuple,[[float(child.attrib['lat']),float(child.attrib['lon'])] for child in gps_root if child.tag == "gps-point"])))

# function to find closest point to current index and sort a path
def findClose(loop) :
    for iter in range(0,loop) :
        for coord in range(0, len(coordinates)):
            low = 10000000000000000
            for nextc in range(coord+1, len(coordinates)):
                if (haversine(coordinates[coord][0],coordinates[coord][1],coordinates[nextc][0],coordinates[nextc][1]) < low): # closest 
                    low = haversine(coordinates[coord][0],coordinates[coord][1],coordinates[nextc][0],coordinates[nextc][1])
                    index = nextc
            coordinates.insert(coord+1, coordinates.pop(index))
        coordinates.reverse() # reverse list since it always almost ends up at the correct finishing point
    
findClose(2) # sort twice
    
# find largest gap between two points and set as first. if your path is fine above, comment this out
large = 0
for i in range(0, len(coordinates)-1):
    if haversine(coordinates[i][0],coordinates[i][1],coordinates[i+1][0],coordinates[i+1][1]) > large:
        large = haversine(coordinates[i][0],coordinates[i][1],coordinates[i+1][0],coordinates[i+1][1])
        index = i
coordinates.insert(0, coordinates.pop(index))

findClose(3) # sort three times

In [59]:
# map reference frame, plot path
LDN_COORDINATES = (33.657948, -117.586311)
myMap = folium.Map(location=LDN_COORDINATES, zoom_start=15)
folium.PolyLine(coordinates,line_opacity = 0.5, weight = 4).add_to(myMap)
colors = [ 'red', 'blue', 'green', 'orange', 'purple', 'beige', 'gray', 'pink', 'black', 'lightgreen', 'darkblue', 'lightblue', 'lightred', 'darkpurple', 'darkred', 'cadetblue', 'lightgray', 'darkgreen' ]

# hacky solution to plot creeps
for x in range(0, len(creeps)): 
    globals()['creep%s' % x] = FeatureGroup(name = 'Creep '+ str(x))
    for creep_loc in creeps[x][1]: Marker(location=creep_loc, popup=creeps[x][0],  icon=folium.Icon(color=colors[x % len(colors)], icon='user-secret', prefix='fa')).add_to(globals()['creep%s' % x])  
    globals()['creep%s' % x].add_to(myMap)
    
# add toggle for creeps
LayerControl().add_to(myMap)
display(myMap)